In [21]:
import pandas as pd
import json

In [4]:
development = pd.read_csv("../data/development.csv")

In [17]:
gpt3Train = pd.DataFrame({"prompt": development["tweet"], "completion": development["ideology_multiclass"]})
gpt3Train["prompt"] = gpt3Train["prompt"] + "->"
gpt3Train["ideology_multiclass"] = gpt3Train["ideology_multiclass"] + '\n'

In [30]:
obj = json.loads(gpt3Train.sample(200).to_json(orient='records'))

In [31]:
with open('gpt3-train.jsonl', 'w') as output:
    for entry in gpt3Train:
        json.dump(entry, output_file)
        output_file.write("\n")